In [12]:
import pandas as pd

frame = pd.read_pickle('/Users/giuliavenditti/Desktop/data.pkl')
frame.head()

,issn,doi,doi-num,on-crossref,reference,asserted-by-cr,asserted-by-pub,ref-undefined,ref-num,year,country,subject
0,2372-0352,10.3934/environsci.2020019,1,1,1,0,52,24,76,2020.0,US,G
1,2372-0352,10.3934/environsci.2020024,1,1,1,0,11,8,19,2020.0,US,G
2,2372-0352,10.3934/environsci.2020026,1,1,1,0,26,23,49,2020.0,US,G
3,2372-0352,10.3934/environsci.2015.3.565,1,1,1,0,37,0,37,2015.0,US,G
4,2372-0352,10.3934/environsci.2021009,1,1,1,21,0,19,40,2021.0,US,G


# 1) DOI on Crossref

In [38]:
import plotly.graph_objects as go
import plotly.express as px

frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
fig = px.bar(frame,x=frame.index,y='perc_cr',
 labels={ "subject": "Research field", "perc_cr": "DOIs on Crossref %" },
 title="Percentage of DOIs registered on Crossref - divided by research field.")

for idx in range(len(fig.data)):
    fig.data[idx].x = ['General works','Philosophy, Psychology, Religion','Auxiliary science of History', 'World History', 'History of the Americs', 'Local History of the Americas', 'Geography, Anthropology, Recreation', 'Social Sciences', 'Political Science', 'Law', 'Education', 'Music', 'Fine Arts', 'Language and Literature', 'Science', 'Medicine', 'Agriculture', 'Technology', 'Military Science', 'Naval Science', 'Bibliography, Library Science']

fig.update_traces(marker_color='rgb(252, 174, 95)', marker_line_color='rgb(252, 174, 95)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
)
fig.show()
# fig.write_html('perc_cr_subj.html')

In [18]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])
fig = px.bar(frame,x=frame.index,y='perc_cr')
fig.show()

In [41]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])
fig = px.bar(frame,x=frame.index,y='perc_cr')
fig.show()

# 2) reference on crosref doi 

In [44]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])
fig = px.bar(frame,x=frame.index,y='perc_ref')
fig.show()

In [22]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])
fig = px.bar(frame,x=frame.index,y='perc_ref')
fig.show()

In [46]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])
fig = px.bar(frame,x=frame.index,y='perc_ref')
fig.show()

# 3) Reference that have doi specified

In [48]:
import plotly.graph_objects as go
import plotly.express as px

frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
frame['perc_ref_doi'] = (frame['ref-num']/frame['ref-undefined'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_ref_doi', 'perc_ref_nodoi'], histfunc='avg')
fig.show()

In [26]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
frame['perc_ref_doi'] = (frame['ref-num']/frame['ref-undefined'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_ref_doi', 'perc_ref_nodoi'], histfunc='avg')
fig.show()

In [51]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
frame['perc_ref_doi'] = (frame['ref-num']/frame['ref-undefined'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_ref_doi', 'perc_ref_nodoi'], histfunc='avg')
fig.show()

# 4) asserted-by crossref / asserted-by publisher

In [13]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_asserted_cr'] = (frame['asserted-by-cr']/frame['ref-num'])*100
frame['perc_asserted_pub'] = (frame['asserted-by-pub']/frame['ref-num'])*100
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_asserted_cr','perc_asserted_pub','perc_ref_nodoi'], histfunc='avg')
fig.show()

In [31]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_asserted_cr'] = (frame['asserted-by-cr']/frame['ref-num'])*100
frame['perc_asserted_pub'] = (frame['asserted-by-pub']/frame['ref-num'])*100
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_asserted_cr','perc_asserted_pub','perc_ref_nodoi'], histfunc='avg')
fig.show()

In [53]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_asserted_cr'] = (frame['asserted-by-cr']/frame['ref-num'])*100
frame['perc_asserted_pub'] = (frame['asserted-by-pub']/frame['ref-num'])*100
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_asserted_cr','perc_asserted_pub','perc_ref_nodoi'], histfunc='avg')
fig.show()